### Setup

First, please make sure you are using a GPU runtime to run this notebook, so inference is much faster. If the following command fails, use the `Runtime` menu above and select `Change runtime type`.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Tue Jan 17 04:09:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    12W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Next, you should install `diffusers==0.10.0` as well `scipy`, `ftfy` and `transformers`. `accelerate` is used to achieve much faster loading.

In [ ]:
!pip install diffusers==0.10.0
!pip install transformers scipy ftfy accelerate
!pip install "ipywidgets>=7,<8"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.9/502.9 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 26.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 116.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.7 MB/s eta 0:00:00


In [ ]:
import torch
from diffusers import StableDiffusionPipeline

# make sure you're logged in with `huggingface-cli login`
pipe = StableDiffusionPipeline.from_pretrained('runwayml/stable-diffusion-v1-5', use_auth_token='<auth_token>')

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Next, let's move the pipeline to GPU to have faster inference.

In [ ]:
pipe = pipe.to("cuda")

And we are ready to generate images:

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
import random 
import numpy as np
import torch
from diffusers import StableDiffusionPipeline

# make sure you're logged in with `huggingface-cli login`
pipe = StableDiffusionPipeline.from_pretrained('runwayml/stable-diffusion-v1-5', use_auth_token='<auth_token>')

pipe = pipe.to("cuda")

plain = '/content/drive/Shareddrives/Visual Metaphors/test_images/SD'
cot = '/content/drive/Shareddrives/Visual Metaphors/test_images/SD-COT'
df = pd.read_csv('/content/final_test_set.csv')

try:
    os.mkdir(plain)
    os.mkdir(cot)
except:
  pass
for i,row in tqdm(df.iterrows()):
  import random
  seed = random.randint(0,4294967295)
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  generator = torch.Generator("cuda").manual_seed(seed)
  prompt_plain = 'An illustration: ' +  row['metaphor']
  prompt_cot = row['davinci2_freq_prompt']
  try:

    os.mkdir(os.path.join(plain, row['metaphor']))
    os.mkdir(os.path.join(cot, row['metaphor']))
  except:
    pass   

  image = pipe(prompt_plain,generator=generator).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)
  # Now to display an image you can either save it such as:
  image.save(os.path.join(plain, row['metaphor'],"SD.jpg"))
  seed = random.randint(0,4294967295)
  generator = torch.Generator("cuda").manual_seed(seed)
  image = pipe(prompt_cot,generator=generator).images[0]  # image here is in [PIL format](https://pillow.readthedocs.io/en/stable/)

  # Now to display an image you can either save it such as:
  image.save(os.path.join(cot, row['metaphor'],"SD_CoT.jpg"))

# or if you're in a google colab you can directly display it with 

100it [00:00, 1142.59it/s]


Running the above cell multiple times will give you a different image every time. If you want deterministic output you can pass a random seed to the pipeline. Every time you use the same seed you'll have the same image result.